# CitiBike TSP
**Algorithms Group 50**\
Donal Lowsley-Williams \
Joan La Rosa \
Isaac Lichter \
Weidong Gao 

We will be conducting the traveling salesman problem on New York City citibike data. We will be using each CitiBike station as the nodes in our graph, and represent the edge costs by the average time it takes to bike from one station to another. Edges will only include those rides in the CitiBike data that have actually occurred - in other words, if no ride in the data started at station A and ended at station B, we will not include that edge. This adds to the realism of the problem as it would be unlikely to take that edge anyways. Moreover, we will only have edges from one station to another - not one station to itself, as there are occurences of rides like this in the CitiBike data.

### Imports
Will we be implementing these algorithms by hand, so we just need libraries for data loading and export, matrix manipulation functions, and some data exploration. 

In [30]:
import csv
import json
import numpy as np
import pandas as pd
import time

# The Data
We will get the data from CitiBike's openly available [System Data](https://s3.amazonaws.com/tripdata/index.html). The data we are using was the most recent as of this notebook's creation date, and was published by CitiBike on [**Oct 4th 2021, 01:26:29 pm**](https://s3.amazonaws.com/tripdata/202109-citibike-tripdata.csv.zip). This is the data of all rides taken on the CitiBike system during the month of September 2021. However, the preprocessing should work for any selection of the data from the system data, since they all should follow the same structure. 

Please store the data in a folder named **"Data"** with the name **"citibike-data.csv"**, like so: **"Data/citibike-data.csv**

In [31]:
df = pd.read_csv('Data/citibike-data.csv',
                 parse_dates=['started_at','ended_at'])

/home/joan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Preprocessing
In order to preprocess the data, 

### Data Cleaning

First lets take a look at the length of our data.

In [32]:
len(df)

3280221

Now, lets see what the column was that caused data type issues. We can see from the error above that it was column 7, which corresponds to the `end_station_id`. 

In [33]:
df.columns[7]

'end_station_id'

In the data, we can see by looking at the csv file that these values are typically float values, so we can cast them to a numeric value. We set any values that can't be cast to a numeric value to `NaN` by setting `errors='coerce'`. 

In [34]:
df['end_station_id'] = df['end_station_id'].apply(pd.to_numeric, errors='coerce')

We drop all null values. (those with `NaN`) This includes not just those with a faulty `end_station_id` value but those with errors elsewhere. We want to use only entries with full and complete data.

In [35]:
df = df.dropna()

We can see that we dropped approx. 15K entries that had data issues or invalid end stations. This is negligible in the grand scheme of the project, given that we have about 3.2M entries. 

In [36]:
len(df)

3264732

Let's take a peek at our data and its types. We can see that many use the dtype object, which pandas uses to refer to strings with varying length. We can leave this be. We have a couple date items, which we set when reading the csv file. The remaining data is all float data. 

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3264732 entries, 0 to 3280220
Data columns (total 13 columns):
ride_id               object
rideable_type         object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name    object
start_station_id      float64
end_station_name      object
end_station_id        float64
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual         object
dtypes: datetime64[ns](2), float64(6), object(5)
memory usage: 348.7+ MB


Let's conduct a quick sanity check to ensure the data is properly cleaned:

In [38]:
len(df['start_station_id'].unique()) == len(df['start_station_name'].unique())

False

Uh-oh. Lets dig into this to see what the problem is. Lets look at these values. 

In [39]:
print('start station id uniques: ',len(df['start_station_id'].unique()))
print('start station name uniques: ',len(df['start_station_name'].unique()))
print(len(df['start_station_id'].unique()) == len(df['start_station_name'].unique()))

start station id uniques:  1488
start station name uniques:  1493
False


So we see that there are more names than there are id's. Lets make a copy of the columns in the data causing an issue. 

In [40]:
sample = df[['start_station_id', 'start_station_name']].copy().drop_duplicates()

Next, we are going to pull each unique row by dropping the duplicates from this sample. This will let us examing all the unique start station id / name pairs. Next, we can count the frequency of each id occuring - since we know there are less id's than there are names. We can then sort this and view the id's that have more than one name.

In [41]:
sample.groupby('start_station_id').count().sort_values('start_station_name').tail(10)

,start_station_name
start_station_id,
5303.08,1
5303.06,1
5300.06,1
5300.05,1
8841.03,1
5382.07,2
4488.09,2
5190.09,2
5422.04,2


Now we see the problematic values are the last five entries in the above dataframe. We can iterate over these and observe what might be the problem by converting them to raw NumPy arrays. 

In [42]:
faulty_ids = np.array(
    sample.groupby('start_station_id').count().sort_values('start_station_name').tail(5).T.columns
)

# iterate over the faulty station ids
for idx, station_id in enumerate(faulty_ids):
    faulty_entries = sample.loc[sample['start_station_id'] == station_id]
    print('Entry',idx,np.array(faulty_entries['start_station_name']))
del sample, faulty_entries, faulty_ids, idx, station_id

Entry 0 ['Forsyth St\\t& Grand St' 'Forsyth St\t& Grand St']
Entry 1 ['Boerum Pl\\t& Pacific St' 'Boerum Pl\t& Pacific St']
Entry 2 ['Clinton St\\t& Cherry St' 'Clinton St\t& Cherry St']
Entry 3 ['Howard St & Lafayette St' 'Howard St & Centre St']
Entry 4 ['Nassau St\\t& Duffield St' 'Nassau St\t& Duffield St']


The problem for most of these appear to be some sort of string parsing issue. This is good, because it means we can ignore it and **simply focus on the station_id's**. However, we see that entry 3 has actual different names. Let us examing this in [Google Maps](https://www.google.com/maps/place/Citi+Bike:+Howard+St+%26+Lafayette+St/@40.7191002,-73.9996465,20z/data=!4m13!1m7!3m6!1s0x89c2598992b42c11:0xa58f248cc38ef667!2sLafayette+St+%26+Howard+St,+New+York,+NY+10013!3b1!8m2!3d40.7192055!4d-73.9998472!3m4!1s0x89c25989ed219c2b:0x5828092994c0b427!8m2!3d40.7191053!4d-73.9997333). We can see that this station is nestled right between those two streets, so it refers to the same station. Again, we can ignore this and **simply focus on the station_id's**.

![title](./lafayettecentre.png)

Let us perform one last sanity check before we start feature engineering, and ensure that there are the same number of start stations as there are end stations.

In [43]:
len(df['start_station_id'].unique()) == len(df['end_station_id'].unique())

True

We can ensure that all the entries in start_station_id are also present in end_station_id. This way, we know by iterating only over one of these lists ensures we are iterating across all potential station_ids. 

In [44]:
is_complete = True
for val in df['start_station_id'].unique():
    if val not in df['end_station_id'].unique():
        is_complete = False
print('data matches') if is_complete else print('Error; data mistmatch')
del val, is_complete

data matches


Looks like we are good to go. Let's pull the necesary data for our algorithms into another dataframe to keep things clean.

### Edge Costs
Now we need to compute edge costs between each station, and store this in a cost matrix. We can do this by first simply subtracting the start time from the end time. Since these are date objects, we can convert them to raw integers representing seconds by dividing by a 1 second numpy date time value to get total seconds and then casting to int32.

In [45]:
df['trip_cost'] = ((df['ended_at'] - df['started_at']) / np.timedelta64(1, 's')).astype(np.int32)

Next up is creating our actual cost data. What we want here is a DataFrame that contains average trip cost from each start station to each end station (when they are not the same), across all rides between those two stations. We can do this by pulling out only our relevant values - `start_station_id`, `end_station_id`, and `trip_cost`, and then dropping all the entries where the start station is equivalent to the end station. Then we group them by their `start_station_id` / `end_station_id` and take an average of the remaining columns - which is just `trip_cost`.

In [46]:
new_df = df[['start_station_id', 'end_station_id', 'trip_cost']]
dropped = new_df.drop(new_df[new_df['start_station_id'] == new_df['end_station_id']].index)
edge_costs = dropped.groupby(by=['start_station_id','end_station_id']).mean().reset_index()
del dropped, new_df;

Lets take a look at our edge cost DataFrame.

In [47]:
edge_costs

,start_station_id,end_station_id,trip_cost
0,2733.03,2782.02,674.857143
1,2733.03,2832.03,1108.142857
2,2733.03,2872.02,651.750000
3,2733.03,2883.03,466.666667
4,2733.03,2912.08,692.333333
...,...,...,...
406129,8841.03,8782.01,1353.250000
406130,8841.03,8795.01,1514.631579
406131,8841.03,8795.03,843.333333
406132,8841.03,8799.01,1213.818182


As we can see, we have each `start_station_id`, each `end_station_id`, and the average `trip_cost` between those stations. We are now prepared to run our algorithms. 

# The Algorithms
Now that we have pulled and preprocessed our data, we can begin to test various algorithms against it. We will test the following algorithms:

**Greedy**
- Nearest Neighbor
- Farthest First Traversal

**?????**
- Christofides Algorithm 

**K-opt Heuristic Algorithms**
- 2-OPT
- 3-OPT
- Lin-Kernighan Heuristic

### Helper Function
Before we do anything, let's write a little function to time our algorithms and make for easy comparison. 

In [ ]:
def check_performance(fn, station_ids, distances):
    start = time.time()
    _, _ = fn(station_ids, distances)
    end = time.time()
    return end - start

def compare_algorithms(fn_list, station_ids, distances, num_iters=5):
    results = {}
    for algo in fn_list:
        t = 0
        for _ in range(num_iters):
            t += check_performance(algo, station_ids, distances)
        results[algo.__name__] = t / num_iters
    return results

## Greedy Algorithms
First, we will examine various greedy algorithms and their effectiveness on the data. 

### Nearest Neighbor

The nearest neighbor algorithm is very simple and simply starts at a random node (or station in our example) and continues to visit unvisited nodes that are as close to the last visited node as possible .

**Psuedocode:** \
These are the steps of the algorithm:

    Initialize all vertices as unvisited.
    Select an arbitrary vertex, set it as the current vertex u. Mark u as visited.
    Find out the shortest edge connecting the current vertex u and an unvisited vertex v.
    Set v as the current vertex u. Mark v as visited.
    If all the vertices in the domain are visited, then terminate. Else, go to step 3.

The sequence of the visited vertices is the output of the algorithm. 

**Time complexity:** Worst Case O(N^2)

**Space complexity:** Worst Case O(N)

[Source (wikipedia)](https://en.wikipedia.org/wiki/Nearest_neighbour_algorithm)

In [ ]:
# Nearest Neighbor
def nearest_neighbor(edge_costs : pd.DataFrame):
    cost = 0
    path = []
    
    return path, cost

### Farthest-First Traversal

**PseudoCode:**\
The farthest-first traversal of a finite point set may be computed by a greedy algorithm that maintains the distance of each point from the previously selected points, performing the following steps:[3]

    Initialize the sequence of selected points to the empty sequence, and the distances of each point to the selected points to infinity.
    While not all points have been selected, repeat the following steps:
        Scan the list of not-yet-selected points to find a point p that has maximum distance from the selected points.
        Remove p from the not-yet-selected points and add it to the end of the sequence of selected points.
        For each remaining not-yet-selected point q, replace the distance stored for q by the minimum of its old value and the distance from p to q.

**Time complexity:** Worst Case O(N^2)

**Space complexity:** Worst Case O(N)

[Source (wikipedia)](https://en.wikipedia.org/wiki/Farthest-first_traversal)

In [ ]:
# Farthest-First Traversal